In [36]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [15]:
symbol = "ADAUSDT"

ADAData = pd.read_csv(f"./preparedData/{symbol}.csv", date_parser = True)
ADAData.head()

,open_time,open,high,low,close,volume,close_time
0,2018-05-01 05:30:00,0.34145,0.34199,0.31870,0.32173,32463784.08,2018-05-01 09:29:59
1,2018-05-01 09:30:00,0.32115,0.34300,0.32000,0.33787,14470581.61,2018-05-01 13:29:59
2,2018-05-01 13:30:00,0.33756,0.35900,0.33421,0.34093,25321414.19,2018-05-01 17:29:59
3,2018-05-01 17:30:00,0.34003,0.34290,0.33166,0.33735,16675168.68,2018-05-01 21:29:59
4,2018-05-01 21:30:00,0.33776,0.35950,0.33714,0.35374,18237446.69,2018-05-02 01:29:59


In [16]:
# number of rows in data
data_shape = ADAData.shape
rows = data_shape[0]

In [24]:
# data train
data_train = ADAData.iloc[ :math.floor(rows*0.8) , :]
data_train = data_train.drop(['close_time', 'open_time'], axis=1)

# data test
data_test = ADAData.iloc[ math.floor(rows*0.8): , :]

print(data_train.shape)
print(data_test.shape)


(6124, 5)
(1532, 7)


In [25]:
# Normalization
# MinMaxScaler is used to normalize the data

scaler = MinMaxScaler()
normalized_data_train = scaler.fit_transform(data_train)
normalized_data_train

array([[0.34200261, 0.33268835, 0.33069698, 0.32087785, 0.04466304],
       [0.32024904, 0.33374473, 0.33212501, 0.33817336, 0.01803671],
       [0.33783407, 0.35047956, 0.34773439, 0.34145244, 0.03409376],
       ...,
       [0.9129857 , 0.90011401, 0.93303674, 0.91704798, 0.10263346],
       [0.91702565, 0.92783106, 0.88691163, 0.89921667, 0.31177294],
       [0.89905484, 0.91008169, 0.89601802, 0.91892326, 0.17351821]])

In [31]:
normalized_data_train = pd.DataFrame(normalized_data_train)
normalized_data_train.columns = ['open', 'high', 'low', 'close', 'volume']

normalized_data_train.head()

,open,high,low,close,volume
0,0.342003,0.332688,0.330697,0.320878,0.044663
1,0.320249,0.333745,0.332125,0.338173,0.018037
2,0.337834,0.350480,0.347734,0.341452,0.034094
3,0.340481,0.333640,0.344933,0.337616,0.021299
4,0.338048,0.351003,0.350953,0.355180,0.023611


In [34]:
X_train = data_train.filter(normalized_data_train.columns.difference(['close']))
Y_train = normalized_data_train['close']

X_test, Y_test = data_test.filter(data_test.columns.difference(['close'])), data_test['close']

X_train, Y_train = np.array(X_train), np.array(Y_train)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(6124, 4) (6124,)
(1532, 6) (1532,)


In [37]:
#Initialize the RNN
model = Sequential() 
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
model.add(Dropout(0.2)) 

model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3)) 

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4)) 

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5)) 

model.add(Dense(units =1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 50)             11200     
_________________________________________________________________
dropout (Dropout)            (None, 4, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 60)             26640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 60)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 80)             45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 80)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)               9

In [38]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

history= model.fit(X_train, Y_train, epochs = 20, batch_size =50, validation_split=0.1)

Epoch 1/20


ValueError: in user code:

    C:\Users\nonit\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\nonit\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\nonit\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\nonit\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\nonit\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)